In [1]:
import pandas as pd

In [3]:
# This code combines all of the three gold standards into one big file

# EA and TM Dataset
perfect_matches_ea_tm = pd.read_csv("ea_tm/perfect_matches_ea_tm.csv")
corner_cases_ea_tm = pd.read_csv("ea_tm/corner_cases_ea_tm.csv")
non_matches_ea_tm = pd.read_csv("ea_tm/non_matches_ea_tm.csv")

# FM and EA Dataset
perfect_matches_fm_ea = pd.read_csv("fm_ea/perfect_matches_fm_ea_v2.csv")
corner_cases_fm_ea = pd.read_csv("fm_ea/corner_cases_fm_ea_v2.csv")
non_matches_fm_ea = pd.read_csv("fm_ea/non_matches_fm_ea_v2.csv")

# FM and TM Dataset
perfect_matches_fm_tm = pd.read_csv("fm_tm/perfect_matches_fm_tm_v3.csv")
corner_cases_fm_tm = pd.read_csv("fm_tm/corner_cases_fm_tm_v3.csv")
non_matches_fm_tm = pd.read_csv("fm_tm/non_matches_fm_tm_v3.csv")

# perfect_matches
perfect_matches = pd.concat([
    perfect_matches_ea_tm,
    perfect_matches_fm_ea,
    perfect_matches_fm_tm
])

# corner_cases
corner_cases = pd.concat([
    corner_cases_ea_tm,
    corner_cases_fm_ea,
    corner_cases_fm_tm
])

# non_matches
non_matches = pd.concat([
    non_matches_ea_tm,
    non_matches_fm_ea,
    non_matches_fm_tm
])


# Concat all of the datasets
gold_standard = pd.concat([perfect_matches, corner_cases, non_matches])

number_extract_perfect_matches = 100
number_extract_corner_cases = 200
number_extract_non_matches = 200

# Gold Standard ea_tm
gold_standard_ea_tm = pd.concat([
    perfect_matches_ea_tm.sample(n=number_extract_perfect_matches, random_state=42), 
    corner_cases_ea_tm.sample(n=number_extract_corner_cases, random_state=42), 
    non_matches_ea_tm.sample(n=number_extract_non_matches, random_state=42)
    ])

# Gold Standard fm_ea
gold_standard_fm_ea = pd.concat([
    perfect_matches_fm_ea.sample(n=number_extract_perfect_matches, random_state=42), 
    corner_cases_fm_ea.sample(n=number_extract_corner_cases, random_state=42), 
    non_matches_fm_ea.sample(n=number_extract_non_matches, random_state=42)
    ])

# Gold Standard fm_tm
gold_standard_fm_tm = pd.concat([
    perfect_matches_fm_tm.sample(n=number_extract_perfect_matches, random_state=42), 
    corner_cases_fm_tm.sample(n=number_extract_corner_cases, random_state=42), 
    non_matches_fm_tm.sample(n=number_extract_non_matches, random_state=42)
    ])

In [4]:
len(gold_standard_fm_tm)

500

In [20]:
gold_standard_ea_tm.iloc[:, [0, 1, -1]].to_csv("gold_standard_ea_tm.csv", index=False)
gold_standard_fm_ea.iloc[:, [0, 1, -1]].to_csv("gold_standard_fm_ea.csv", index=False)
# gold_standard_fm_tm.iloc[:, [0, 1, -1]].to_csv("gold_standard_fm_tm_v3.csv", index=False)

### Following code added by Jan
to produce gold standard training sets to the 500 entity test sets to ensure all models (ML and linear) are tested on same data

In [6]:
# fm_tm
# Get the indices of the sampled rows
sampled_indices_pm = perfect_matches_fm_tm.sample(n=number_extract_perfect_matches, random_state=42).index
sampled_indices_cc = corner_cases_fm_tm.sample(n=number_extract_corner_cases, random_state=42).index
sampled_indices_nm = non_matches_fm_tm.sample(n=number_extract_non_matches, random_state=42).index

# Get the rows that were not sampled
remaining_pm = perfect_matches_fm_tm.drop(sampled_indices_pm)
remaining_cc = corner_cases_fm_tm.drop(sampled_indices_cc)
remaining_nm = non_matches_fm_tm.drop(sampled_indices_nm)

# Concatenate the remaining rows in ratio of 1:2:2 (DEFINE FUNCTIONS IN CODE BELOW FIRST!)
gold_standard_fm_tm_train_NEW = create_goldstandard_balance_1_2_2(remaining_pm, remaining_cc, remaining_nm)

150.0 300 300


In [12]:
# fm_ea
# Get the indices of the sampled rows
sampled_indices_pm = perfect_matches_fm_ea.sample(n=number_extract_perfect_matches, random_state=42).index
sampled_indices_cc = corner_cases_fm_ea.sample(n=number_extract_corner_cases, random_state=42).index
sampled_indices_nm = non_matches_fm_ea.sample(n=number_extract_non_matches, random_state=42).index

# Get the rows that were not sampled
remaining_pm = perfect_matches_fm_ea.drop(sampled_indices_pm)
remaining_cc = corner_cases_fm_ea.drop(sampled_indices_cc)
remaining_nm = non_matches_fm_ea.drop(sampled_indices_nm)

# Concatenate the remaining rows in ratio of 1:2:2 (DEFINE FUNCTIONS IN CODE BELOW FIRST!)
gold_standard_fm_ea_train_NEW = create_goldstandard_balance_1_2_2(remaining_pm, remaining_cc, remaining_nm)

28 56 56


In [17]:
# ea_tm
# Get the indices of the sampled rows
sampled_indices_pm = perfect_matches_ea_tm.sample(n=number_extract_perfect_matches, random_state=42).index
sampled_indices_cc = corner_cases_ea_tm.sample(n=number_extract_corner_cases, random_state=42).index
sampled_indices_nm = non_matches_ea_tm.sample(n=number_extract_non_matches, random_state=42).index

# Get the rows that were not sampled
remaining_pm = perfect_matches_ea_tm.drop(sampled_indices_pm)
remaining_cc = corner_cases_ea_tm.drop(sampled_indices_cc)
remaining_nm = non_matches_ea_tm.drop(sampled_indices_nm)

# Concatenate the remaining rows in ratio of 1:2:2 (DEFINE FUNCTIONS IN CODE BELOW FIRST!)
remaining_cc_sampled = remaining_cc.sample(n=216, random_state=42)
gold_standard_ea_tm_train_NEW = pd.concat([remaining_pm, remaining_cc_sampled, remaining_nm])
print(len(remaining_pm), len(remaining_cc_sampled), len(remaining_nm))
# # of non-matches is too small (16) 
# compensate by adding 100 from corner cases
# since perfect matches are 58 (58*2 = 116 and there is already 16 non-matches) so its balanced again

58 216 16


In [18]:
print(len(gold_standard_fm_tm_train_NEW))
print(len(gold_standard_fm_ea_train_NEW))
print(len(gold_standard_ea_tm_train_NEW))

750
140
290


In [19]:
gold_standard_fm_tm_train_NEW.iloc[:, [0, 1, -1]].to_csv("gold_standard_fm_tm_train_NEW.csv", index=False)
gold_standard_fm_ea_train_NEW.iloc[:, [0, 1, -1]].to_csv("gold_standard_ea_fm_train_NEW.csv", index=False)
gold_standard_ea_tm_train_NEW.iloc[:, [0, 1, -1]].to_csv("gold_standard_ea_tm_train_NEW.csv", index=False)

In [10]:
def print_information(df, name):
    overall_number_matches = sum(df['match'] == True)
    overall_number_nonmatches = len(df) - overall_number_matches

    print("-"*5, name, "-"*5)
    print(f"Number of rows: {len(df)}")
    print(f"Number of matches: {overall_number_matches}, {round(overall_number_matches/len(df)*100, 2)}%")
    print(f"Number of non-matches: {overall_number_nonmatches}, {round(overall_number_nonmatches/len(df)*100, 2)}%")
    print()

    # df.head()

In [5]:
# Save gold standard to disk, only the keys and the match column
gold_standard.iloc[:, [0, 1, -1]].to_csv("gold_standard.csv", index=False)

In [11]:
print_information(gold_standard, "Gold Standard")

print_information(perfect_matches_ea_tm, "Perfect Matches EA and TM")
print_information(corner_cases_ea_tm, "Corner Cases EA and TM")
print_information(non_matches_ea_tm, "Non Matches EA and TM")

print_information(perfect_matches_fm_ea, "Perfect Matches FM and EA")
print_information(corner_cases_fm_ea, "Corner Cases FM and EA")
print_information(non_matches_fm_ea, "Non Matches FM and EA")

print_information(perfect_matches_fm_tm, "Perfect Matches FM and TM")
print_information(corner_cases_fm_tm, "Corner Cases FM and TM")
print_information(non_matches_fm_tm, "Non Matches FM and TM")

----- Gold Standard -----
Number of rows: 4968
Number of matches: 3357, 67.57%
Number of non-matches: 1611, 32.43%

----- Perfect Matches EA and TM -----
Number of rows: 158
Number of matches: 158, 100.0%
Number of non-matches: 0, 0.0%

----- Corner Cases EA and TM -----
Number of rows: 666
Number of matches: 527, 79.13%
Number of non-matches: 139, 20.87%

----- Non Matches EA and TM -----
Number of rows: 216
Number of matches: 0, 0.0%
Number of non-matches: 216, 100.0%

----- Perfect Matches FM and EA -----
Number of rows: 128
Number of matches: 128, 100.0%
Number of non-matches: 0, 0.0%

----- Corner Cases FM and EA -----
Number of rows: 356
Number of matches: 264, 74.16%
Number of non-matches: 92, 25.84%

----- Non Matches FM and EA -----
Number of rows: 396
Number of matches: 0, 0.0%
Number of non-matches: 396, 100.0%

----- Perfect Matches FM and TM -----
Number of rows: 2048
Number of matches: 2048, 100.0%
Number of non-matches: 0, 0.0%

----- Corner Cases FM and TM -----
Number 

# Create Train-Test-Splits

In [5]:
from sklearn.model_selection import train_test_split

def create_goldstandard_train_test_split(df, test_size=0.2, min_train_rows=500):
    """Create a train-test split of the goldstandard data.

    Args:
        df (pd.DataFrame): The goldstandard data.
        test_size (float): The proportion of the data to be used for testing.
        min_train_rows (int): The minimum number of rows in the training dataset.

    Returns:
        pd.DataFrame: The training data.
        pd.DataFrame: The testing data.
    """
    # Check if the minimum number of rows in the training dataset is less than the total number of rows
    if min_train_rows >= len(df):
        print(f"The minimum number of rows in the training dataset should be less than the total number of rows. Dataset has {len(df)} rows. Minimum number of rows in the training dataset is {min_train_rows}.")

    # Perform the train-test split
    train, test = train_test_split(df, test_size=test_size, random_state=42)
    return train, test


def create_goldstandard_balance_1_2_2(perfect_matches:pd.DataFrame, corner_cases:pd.DataFrame, non_matches:pd.DataFrame) -> pd.DataFrame:

    num_perfect_matches = len(perfect_matches)
    num_corner_cases    = len(corner_cases)
    num_non_matches     = len(non_matches)

    num_corner_cases = min(num_corner_cases, num_non_matches)
    num_non_matches  = min(num_corner_cases, num_non_matches)

    if num_corner_cases < (2 * num_perfect_matches):
        num_perfect_matches = num_corner_cases / 2
    else:
        num_corner_cases = 2 * num_perfect_matches
        num_non_matches  = 2 * num_perfect_matches
    print(num_perfect_matches, num_corner_cases, num_non_matches)

    # Gold Standard
    gold_standard = pd.concat([
        perfect_matches.sample(n=int(num_perfect_matches), random_state=42), 
        corner_cases.sample(n=int(num_corner_cases), random_state=42), 
        non_matches.sample(n=int(num_non_matches), random_state=42)
        ])
    
    return gold_standard

In [14]:
gold_standards = [
    # 'ea_tm',
    # 'fm_ea',
    'fm_tm'
]

In [19]:
version = "_v3"
for gold_standard in gold_standards:
    df_perfect_matches = pd.read_csv(f"{gold_standard}/perfect_matches_{gold_standard}{version}.csv")
    df_corner_cases = pd.read_csv(f"{gold_standard}/corner_cases_{gold_standard}{version}.csv")
    df_non_matches = pd.read_csv(f"{gold_standard}/non_matches_{gold_standard}{version}.csv")

    print_information(df_perfect_matches, "df_perfect_matches")
    print_information(df_corner_cases,  "df_corner_cases")
    print_information(df_non_matches, "df_non_matches")

    df_balanced = create_goldstandard_balance_1_2_2(df_perfect_matches, df_corner_cases, df_non_matches)
    train, test = create_goldstandard_train_test_split(df_balanced)

    print_information(train, "Training")
    print_information(test, "Test")

    train.to_csv(f"gold_standard_{gold_standard}_train{version}.csv", index=False)
    test.to_csv(f"gold_standard_{gold_standard}_test{version}.csv", index=False)

----- df_perfect_matches -----
Number of rows: 2048
Number of matches: 2048, 100.0%
Number of non-matches: 0, 0.0%

----- df_corner_cases -----
Number of rows: 500
Number of matches: 232, 46.4%
Number of non-matches: 268, 53.6%

----- df_non_matches -----
Number of rows: 500
Number of matches: 0, 0.0%
Number of non-matches: 500, 100.0%

250.0 500 500
----- Training -----
Number of rows: 1000
Number of matches: 380, 38.0%
Number of non-matches: 620, 62.0%

----- Test -----
Number of rows: 250
Number of matches: 102, 40.8%
Number of non-matches: 148, 59.2%



In [20]:
for gold_standard in gold_standards:
    print("-"*5, gold_standard, "-"*5)
    try:
        pd.read_csv(f"gold_standard_{gold_standard}_train{version}.csv")[["df1", "df2", "match"]].to_csv(f"gold_standard_{gold_standard}_train{version}.csv", index=False)
        pd.read_csv(f"gold_standard_{gold_standard}_test{version}.csv")[["df1", "df2", "match"]].to_csv(f"gold_standard_{gold_standard}_test{version}.csv", index=False)
    except:
        continue

----- fm_tm -----
